# 1. Producing the data  
In this task, we will implement Apache Kafka producers to simulate real-time data streaming. Spark and parallel data processing should not be used in this section, as we are simulating sensors that often lack processing capabilities.  

1.	Every 5 seconds, load 5 days of weather data from the CSV file. We refer to this as weather5s to explain the tasks; feel free to use your own variable name. You should keep a pointer in the file reading process and advance it per read. The data reading should be in chronological order.
2.	Add the current timestamp (weather_ts) to the weather5s and spread your batch out evenly for 5 seconds for each day. Since the weather data is hourly readings, each day you shall have 24 records (120 records in total for 5 days).
For example, assume you send the records at 2025-01-26 00:00:00 (ISO format: YYYY-MM-DD HH:MM:SS) -> (ts = 1737810000):  
Day 1(records 1-24): ts = 1737810000  
Day 2(records 25-48): ts = 1737810001  
Day 3(records 49-72): ts = 1737810002  
…
3.	Send your batch of weather data to a Kafka topic with an appropriate name.




In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import datetime
import csv
import io
import json

#configuration
hostip = "10.192.90.63" #change to your machine IP address

topic = 'weather_data'


def publish_message(producer_instance, topic_name, key, value):
    try:
        print("\nBegin publish")
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: \n' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

def read_csv_dict_chunks(filepath, chunk_size):
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        while True:
            chunk = []
            for _ in range(chunk_size):
                try:
                    row = next(reader)
                    # Add timestamp
                    row["weather_ts"] = datetime.now().isoformat()
                    chunk.append(row)   # each row is already a dict
                except StopIteration:
                    break
            if not chunk:
                break
            yield chunk

def chunk_to_string(chunk):
    return json.dumps(chunk, ensure_ascii=False)

def read_csv_in_chunks(filepath, chunk_size):
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        headers = next(reader)  # Read header row once
        headers.append("weather_ts")
        while True:
            chunk = []
            now = datetime.now().isoformat() # Same timestamp for all items in chunk
            for _ in range(chunk_size):
                try:
                    row = next(reader)
#                     row.append(datetime.now().isoformat()) # Timestamp for read time of each line
                    row.append(now) 
                    chunk.append(row)
                except StopIteration:
                    break
            if not chunk:
                break
            yield headers, chunk
            
def rows_to_csv_string(rows):
    buf = io.StringIO()
    writer = csv.writer(buf)
    writer.writerows(rows)
    return buf.getvalue().strip()

def write_dict_chunk_to_csv(filepath, headers, rows):
    with open(filepath, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        # write header only once if the file is empty
        if f.tell() == 0:
            writer.writeheader()
        writer.writerows(rows)

if __name__ == '__main__':   
    producer = connect_kafka_producer()
    for rows in read_csv_dict_chunks("data/weather.csv", chunk_size=120):
        data = chunk_to_string(rows)
        # data = rows_to_csv_string(rows)
        publish_message(producer, topic, 'parsed', data)
        sleep(5)


Begin publish
Message published successfully. Data: 
0,2022-01-01 22:00:00.000,26.7,,18.3,1016.9,230.0,3.1,2025-10-07T10:37:53.303805
0,2022-01-01 23:00:00.000,25.6,,18.3,1017.5,230.0,3.1,2025-10-07T10:37:53.303805
0,2022-01-02 00:00:00.000,24.4,6.0,18.9,1018.1,270.0,2.6,2025-10-07T10:37:53.303805
0,2022-01-02 01:00:00.000,23.9,4.0,18.3,1018.5,300.0,2.1,2025-10-07T10:37:53.303805
0,2022-01-02 02:00:00.000,22.2,,19.4,,360.0,5.7,2025-10-07T10:37:53.303805
0,2022-01-02 03:00:00.000,21.1,,18.9,1019.5,20.0,5.1,2025-10-07T10:37:53.303805
0,2022-01-02 04:00:00.000,20.6,,17.8,1019.4,30.0,4.6,2025-10-07T10:37:53.303805
0,2022-01-02 05:00:00.000,19.4,4.0,17.2,1019.3,20.0,2.6,2025-10-07T10:37:53.303805
0,2022-01-02 06:00:00.000,18.9,6.0,17.2,1019.0,10.0,2.1,2025-10-07T10:37:53.303805
0,2022-01-02 07:00:00.000,18.9,,17.2,1018.4,10.0,2.6,2025-10-07T10:37:53.303805
0,2022-01-02 08:00:00.000,18.9,,16.7,1018.5,360.0,3.6,2025-10-07T10:37:53.303805
0,2022-01-02 09:00:00.000,18.3,,16.7,1018.1,10.0,3.1,20

Begin publish
Message published successfully. Data: 
0,2022-01-06 22:00:00.000,17.2,,13.9,1018.6,350.0,6.2,2025-10-07T10:37:58.331110
0,2022-01-06 23:00:00.000,16.1,,14.4,1018.5,360.0,6.7,2025-10-07T10:37:58.331110
0,2022-01-07 00:00:00.000,15.6,8.0,13.9,1019.0,360.0,6.7,2025-10-07T10:37:58.331110
0,2022-01-07 01:00:00.000,15.6,,13.3,1019.0,360.0,7.7,2025-10-07T10:37:58.331110
0,2022-01-07 02:00:00.000,15.0,,13.3,1019.6,360.0,6.7,2025-10-07T10:37:58.331110
0,2022-01-07 03:00:00.000,14.4,,13.3,1019.5,360.0,5.1,2025-10-07T10:37:58.331110
0,2022-01-07 04:00:00.000,14.4,,13.3,1019.5,340.0,5.1,2025-10-07T10:37:58.331110
0,2022-01-07 05:00:00.000,14.4,,13.3,1018.9,340.0,4.6,2025-10-07T10:37:58.331110
0,2022-01-07 06:00:00.000,14.4,8.0,13.3,1018.4,330.0,4.1,2025-10-07T10:37:58.331110
0,2022-01-07 07:00:00.000,14.4,,13.3,1018.0,340.0,4.1,2025-10-07T10:37:58.331110
0,2022-01-07 08:00:00.000,13.9,,13.3,1017.4,340.0,4.1,2025-10-07T10:37:58.331110
0,2022-01-07 09:00:00.000,13.9,,13.3,1016.7,360.0,

KeyboardInterrupt: 

In [ ]:
# GPT
import csv

def read_csv_in_chunks(filepath, chunk_size):
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        headers = next(reader)  # Read header row once
        while True:
            chunk = []
            for _ in range(chunk_size):
                try:
                    row = next(reader)
                    chunk.append(row)
                except StopIteration:
                    break
            if not chunk:
                break
            yield headers, chunk
            
for headers, rows in read_csv_in_chunks("large_file.csv", chunk_size=1000):
    print(f"Read {len(rows)} rows")
    # process rows here

